# Item-Based Collaborative Filtering

There are multiple ways that we can use collaborative filtering. One approach would be to look at the similarities between users, called the neighbourhood-bassed approach. For example, in the neighborhood-based approach a number of users are selected based on their similarity to the chosen user. A prediction for the chosen user is made by calculating a weighted average of the ratings of the selected users. 

Another approach is the item-based approach where the ratings are used to measure the correlation between items. The correlation score can be used as a measure. We'll be building a full blown recommender system using the item-based approach that looks at the movie ratings every user gave in order to recommend them movies they are most likely to love.

In [1]:
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('/Users/jacquesthibodeau/Python Data Science/ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3))

m_cols = ['movie_id', 'title']
movies = pd.read_csv('/Users/jacquesthibodeau/Python Data Science/ml-100k/u.item', encoding = "ISO-8859-1", sep='|', names=m_cols, usecols=range(2))

ratings = pd.merge(movies, ratings)

ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


Let's pivot the table and make each row contain all of the information for a specific user in order to get a better view of our dataset.

In [7]:
userRatings = ratings.pivot_table(index=['user_id'], columns=['title'], values=['rating'])
userRatings.head()

rating                                     \
title   'Til There Was You (1997) 1-900 (1994) 101 Dalmatians (1996)   
user_id                                                                
0                             NaN          NaN                   NaN   
1                             NaN          NaN                   2.0   
2                             NaN          NaN                   NaN   
3                             NaN          NaN                   NaN   
4                             NaN          NaN                   NaN   

                                                                    \
title   12 Angry Men (1957) 187 (1997) 2 Days in the Valley (1996)   
user_id                                                              
0                       NaN        NaN                         NaN   
1                       5.0        NaN                         NaN   
2                       NaN        NaN                         NaN   
3                       NaN        2.0                         NaN   
4                       NaN        NaN                         NaN   

                                                                          \
title   20,000 Leagues Under the Sea (1954) 2001: A Space Odyssey (1968)   
user_id                                                                    
0                                       NaN                          NaN   
1                                       3.0                          4.0   
2                                       NaN                          NaN   
3                                       NaN                          NaN   
4                                       NaN                          NaN   

                                                                          \
title   3 Ninjas: High Noon At Mega Mountain (1998) 39 Steps, The (1935)   
user_id                                                                    
0                                               NaN                  NaN   
1                                               NaN                  NaN   
2                                               1.0                  NaN   
3                                               NaN                  NaN   
4                                               NaN                  NaN   

                       ...                                     \
title                  ...                 Yankee Zulu (1994)   
user_id                ...                                      
0                      ...                                NaN   
1                      ...                                NaN   
2                      ...                                NaN   
3                      ...                                NaN   
4                      ...                                NaN   

                                                      \
title   Year of the Horse (1997) You So Crazy (1994)   
user_id                                                
0                            NaN                 NaN   
1                            NaN                 NaN   
2                            NaN                 NaN   
3                            NaN                 NaN   
4                            NaN                 NaN   

                                                                          \
title   Young Frankenstein (1974) Young Guns (1988) Young Guns II (1990)   
user_id                                                                    
0                             NaN               NaN                  NaN   
1                             5.0               3.0                  NaN   
2                             NaN               NaN                  NaN   
3                             NaN               NaN                  NaN   
4                             NaN               NaN                  NaN   

                                                                               \
title   You

NaN means that the user has not yet rated a particular movie. When building a recommender system, we need to have an idea how different items relate to each other. We can easily understand the relationship between movies by looking at the correlation score of those two movies. The score essentially tells us that if at least one user has seen both movies, it will tell us whether 

In [8]:
corrMatrix = userRatings.corr()
corrMatrix.head()

rating               \
title                            'Til There Was You (1997) 1-900 (1994)   
       title                                                              
rating 'Til There Was You (1997)                       1.0          NaN   
       1-900 (1994)                                    NaN          1.0   
       101 Dalmatians (1996)                          -1.0          NaN   
       12 Angry Men (1957)                            -0.5          NaN   
       187 (1997)                                     -0.5          NaN   

                                                                            \
title                            101 Dalmatians (1996) 12 Angry Men (1957)   
       title                                                                 
rating 'Til There Was You (1997)             -1.000000           -0.500000   
       1-900 (1994)                                NaN                 NaN   
       101 Dalmatians (1996)                  1.000000           -0.049890   
       12 Angry Men (1957)                   -0.049890            1.000000   
       187 (1997)                             0.269191            0.666667   

                                                                         \
title                            187 (1997) 2 Days in the Valley (1996)   
       title                                                              
rating 'Til There Was You (1997)  -0.500000                    0.522233   
       1-900 (1994)                     NaN                         NaN   
       101 Dalmatians (1996)       0.269191                    0.048973   
       12 Angry Men (1957)         0.666667                    0.256625   
       187 (1997)                  1.000000                    0.596644   

                                                                      \
title                            20,000 Leagues Under the Sea (1954)   
       title                                                           
rating 'Til There Was You (1997)                                 NaN   
       1-900 (1994)                                              NaN   
       101 Dalmatians (1996)                                0.266928   
       12 Angry Men (1957)                                  0.274772   
       187 (1997)                                                NaN   

                                                               \
title                            2001: A Space Odyssey (1968)   
       title                                                    
rating 'Til There Was You (1997)                    -0.426401   
       1-900 (1994)                                 -0.981981   
       101 Dalmatians (1996)                        -0.043407   
       12 Angry Men (1957)                           0.178848   
       187 (1997)                                   -0.554700   

                                                                              \
title                            3 Ninjas: High Noon At Mega Mountain (1998)   
       title                                                                   
rating 'Til There Was You (1997)                                         NaN   
       1-900 (1994)                                                      NaN   
       101 Dalmatians (1996)                                             NaN   
       12 Angry Men (1957)                                               NaN   
       187 (1997)                                                        NaN   

                                                       \
title                            39 Steps, The (1935)   
       title                                            
rating 'Til There Was You (1997)                  NaN   
       1-900 (1994)                               NaN   
       101 Dalmatians (1996)                 0.111111   
       12 Angry Men (1957)                   0.457176   
       187 (1997)                            1.000000   

                             